In [1]:
from smach_utils2 import *
import matplotlib.pyplot as plt
#arm = moveit_commander.MoveGroupCommander('arm')


/home/roboworks/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [ ]:
omni_base.tiny_move( velX=1.0,std_time=0.2, MAX_VEL=0.3) 

In [ ]:
gripper.steady()

In [ ]:
gripper.close(0.0061)

In [ ]:
arm=moveit_commander.MoveGroupCommander('arm')

In [ ]:

whole_body=moveit_commander.MoveGroupCommander('whole_body_weighted')

In [ ]:
pose=whole_body.get_current_pose()
pose.pose.position.x,pose.pose.position.y, pose.pose.position.z

In [ ]:
whole_body.set_joint_value_target

In [ ]:
wb_v=whole_body.get_current_joint_values()
wb_v[3]+=0.1
whole_body.set_joint_value_target(wb_v)

In [ ]:
res = omni_base.move_base(known_location='place_shelf', time_out=200)
high_shelf_place=np.asarray([0.15050079265248406,
 -0.5861087653247212,
 0.0850017715788356,
 -0.9342371117035135,
 0.022035143687695324,
 0.0])
mid_shelf_place=np.asarray([0.2556342351169577,
 -1.6040815280188854,
 -0.008078345343693893,
 -0.05795663028496589,
 0.0019971082238159354,
 0.0])

low_shelf_place=np.asarray([0.04577467546491997,
 -2.262541528018885,
 0.00010865465630605087,
 0.701618369715034,
 -0.0003068917761841483,
 0.0]
)

In [ ]:
head.set_joint_values([-0.5,-0.65])
arm.set_named_target('go')
arm.go()

In [ ]:

rospy.sleep(0.6)
rospy.sleep(0.6)
image= cv2.cvtColor(rgbd.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)

In [ ]:

plt.imshow(debug_image)

In [ ]:
request= segmentation_server.request_class() 



request.height.data=-1  #MID SHELF FOR PLACING 


res=placing_finder_server.call(request)
#succ=seg_res_tf(res)
print (f'Placing Area at {res.poses.data}')
tf_man.pub_static_tf(pos=[res.poses.data[0], res.poses.data[1],res.poses.data[2]], rot =[0,0,0,1], point_name='placing_area')
img=bridge.imgmsg_to_cv2(res.im_out.image_msgs[0])
#cv2.imshow('our of res'  , img)
plt.imshow(img)

In [ ]:
res = omni_base.move_base(known_location='place_shelf', time_out=12)


In [ ]:
base_grasp_D(tf_name='placing_area',d_x=0.76, d_y=0.0,timeout=30)
arm.go (mid_shelf_place)
base_grasp_D(tf_name='placing_area',d_x=0.6, d_y=0.0,timeout=30)
gripper.open()

In [ ]:
arm.go (mid_shelf_place)

In [ ]:
base_grasp_D(tf_name='placing_area',d_x=0.6, d_y=0.0,timeout=30)
gripper.open()

In [ ]:

for i in range(len(res.poses)):
    tf_man.getTF("head_rgbd_sensor_rgb_frame")
    tf_man.pub_static_tf(pos=[res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z], rot=[0,0,0,1],ref="head_rgbd_sensor_rgb_frame",point_name=res.names[i].data[4:] )   
    rospy.sleep(0.3)    tf_man.change_ref_frame_tf(res.names[i].data[4:])



In [ ]:
omni_base.move_d_to(target_distance=0.78,target_link='apple')

In [ ]:
target_object='apple'
trans,_=tf_man.getTF(target_object,ref_frame="base_link")
trans


In [ ]:
#x,y,z= 5.8 , 1.6, 0.47   #SIM TMR
#quat=[0.0,0.0,0.0,1.0]
x,y,z,=-0.9,-0.7,0
quat=[0.0,0.0,0.0,1.0]

#x,y,z= -0.522 , -2.84, 0.8   #REAL LAB
#quat=[0.0,0.0,0.707,-0.707]
tf_man.pub_static_tf(pos=[x,y,z], rot=quat,point_name='shelf') ### Ideal  coordinates to place Bowl



In [ ]:
line_up_TF()
move_D_to(tf_name='shelf')

In [ ]:

def line_up_TF(tf_name='placing_area', timeout=30.0):
    pose,rot= tf_man.getTF("base_link",ref_frame=tf_name)
    delta_th=tf.transformations.euler_from_quaternion(rot)[2]
    print (pose[1], delta_th)
    timeout = rospy.Time.now().to_sec() + timeout
    active= True
    while (active and timeout >= rospy.Time.now().to_sec()) and not rospy.is_shutdown():
        
        if (abs(pose[1])<0.01 and abs(delta_th)<0.05 ): active = False
        pose,rot= tf_man.getTF("base_link",ref_frame=tf_name)
        delta_th=tf.transformations.euler_from_quaternion(rot)[2]
        print (pose[1], delta_th, active)
        
        if (abs(delta_th)>0.1):pose[1]=0
        
        omni_base.tiny_move( velX=0.0,velY=-pose[1], velT=-delta_th,std_time=0.2, MAX_VEL=0.3) 
    return (abs(pose[1])<0.01 and abs(delta_th)<0.1 )


In [ ]:
tf_name='placing_area'
pose,rot= tf_man.getTF("base_link",ref_frame=tf_name)
delta_th=tf.transformations.euler_from_quaternion(rot)[2]
print (delta_th)

In [ ]:

def new_move_D_to(tf_name='placing_area',d_x=15 , timeout=30.0):
#def base_grasp_D(tf_name,d_x=0.66,d_y=-0.1,timeout=1.0):
    timeout = rospy.Time.now().to_sec() + timeout
    
    
   
    
    succ = False            
    i=0
    while (not succ and not rospy.is_shutdown():
        
        rob_pos,rot=tf_man.getTF('base_link')    
        robot_yaw=tf.transformations.euler_from_quaternion(rot)[2]
        pose,_= tf_man.getTF("base_link",ref_frame=tf_name)
        target_yaw = math.atan2(pose[1], pose[0])+np.pi
        delta_th=   target_yaw-robot_yaw
        delta_th = (delta_th + np.pi) % (2 * np.pi) - np.pi
        
        print("robotyaw {:.2f}, , target_yaw {:.2f}, deltha th {:.2f}".format(robot_yaw, target_yaw,delta_th))
        i+=1                
        #trans
        eX = np.linalg.norm((pose[0:2]))
        eX+= -d_x  #x offest
        velX= eX 
        rospy.loginfo("error_D: {:.2f}, , delta_th {:.2f}, target obj frame {}".format(eX,  delta_th,tf_name))
        
        
        print("error_D: {:.2f}, , delta_th {:.2f}, target obj frame {}, D to target{:.2f}".format(eX,  delta_th,tf_name,np.linalg.norm((pose[0:2]))))  
    
        if abs(delta_th)>=0.1:
            velX=0
       
        succ =  eX <= 0.1  and abs(delta_th)<=0.1
        
        corr_velX = max(min(velX, 0.051), -0.051)
        
        
       
        print("error_D: {:.2f}, , delta_th {:.2f}, target obj frame {}".format(eX,  delta_th,tf_name))
        print (corr_velX)
       
        omni_base.tiny_move( velX=corr_velX,velY=0, velT=delta_th,std_time=0.2, MAX_VEL=0.3) 
    return succ


In [ ]:
new_move_D_to(tf_name='shelf',d_x=0.5)

In [ ]:
def new_move_D_to(tf_name='placing_area',d_x=15 , timeout=30.0):

    timeout = rospy.Time.now().to_sec() + timeout
    succ = False            
    i=0
    while (timeout >= rospy.Time.now().to_sec()) and not succ and not rospy.is_shutdown():
        
        _,rot=tf_man.getTF('base_link')    
        robot_yaw=tf.transformations.euler_from_quaternion(rot)[2]
        pose,_= tf_man.getTF("base_link",ref_frame=tf_name)
        target_yaw = math.atan2(pose[1], pose[0])+np.pi
        delta_th=   target_yaw-robot_yaw
        delta_th = (delta_th + np.pi) % (2 * np.pi) - np.pi
        i+=1                
        eX = np.linalg.norm((pose[0:2]))
        eX+= -d_x  #x offest
        velX= eX 
        if abs(delta_th)>=0.1:velX=0
        succ =  eX <= 0.1  and abs(delta_th)<=0.1
        corr_velX = max(min(velX, 0.051), -0.051)
        #if i >=10:
        print("error_D: {:.2f}, , delta_th {:.2f}, target obj frame {}, D to target{:.2f}".format(eX,  delta_th,tf_name,np.linalg.norm((pose[0:2]))))  
        #    i=0
        omni_base.tiny_move( velX=corr_velX,velY=0, velT=delta_th,std_time=0.2, MAX_VEL=0.3) 
    return succ

#------------------------------------------------------

In [ ]:
omni_base.tiny_move( velX=0.0,velY=-pose[1], velT=-delta_th,std_time=0.2, MAX_VEL=0.3) a

In [ ]:
target_object='apple'
_,rot= tf_man.getTF("base_link",ref_frame='map')

original_rot=tf.transformations.euler_from_quaternion(rot)[2]
trans,_=tf_man.getTF(target_object,ref_frame="base_link")


trans , original_rot

In [ ]:
_,rot= tf_man.getTF("base_link",ref_frame='map')
original_rot=tf.transformations.euler_from_quaternion(rot)[2]
target_object='apple'


succ = False
            
while not succ:
    
    _,rot= tf_man.getTF("base_link",ref_frame='map')
    trans,_=tf_man.getTF(target_object,ref_frame="base_link")

    trans
    eX, eY, eZ = trans
    
    eX+= -0.4
    eY+= -.06
    
    eT= tf.transformations.euler_from_quaternion(rot)[2] - original_rot #Original 
    print (eT)
    if eT > np.pi: eT=-2*np.pi+eT
    if eT < -np.pi: eT= 2*np.pi+eT
    rospy.loginfo("error: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(eX, eY , eT,target_object))
    X, Y, Z = trans
    rospy.loginfo("Pose: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(X, Y , eT,target_object))
    
    if abs(eX) <=0.05 :
        print ('here')
        eX = 0
    if abs(eY) <=0.05  :
        eY = 0
    if abs(eT   ) < 0.1:
        eT = 0
    succ =  eX == 0 and eY == 0 and eT==0

    
    omni_base.tiny_move( velX=0.2*+eX,velY=0.3*eY, velT=-eT,std_time=0.2, MAX_VEL=0.3) 


In [ ]:
res.names

In [ ]:
hand_cam= HAND_RGB()


In [ ]:
#clear_octo_client()

pickup_pose=[0.65,-1.2,0.0,-1.9, 0.0, 0.0]
succ= arm.go(pickup_pose)
gripper.open()


In [ ]:

image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)
for i in range(len(res.names)):
    if res.names[i].data== '013_apple' or res.names[i].data== '017_orange':
        bbM=res.pt_min.data[2*i:2*i+2]
        bbm=res.pt_max.data[2*i:2*i+2]
        bbx=[bbM,bbm]
print(bbx)
plt.imshow(debug_image)

# [(297, 191), (231, 226)]  at pkcp pose #Real Tak
### [(323, 266), (280, 215)]  at pkcp pose #Gazebo


In [ ]:
eX,eY= 0.3,0.3 

while abs(eX) > 0.05 or abs(eY) > 0.05:
    image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
    img_msg  = bridge.cv2_to_imgmsg(image)
    req      = classify_client.request_class()
    req.in_.image_msgs.append(img_msg)
    res      = classify_client(req)


    for i in range(len(res.names)):
        if res.names[i].data== '013_apple' or res.names[i].data== '017_orange':
            bbM=res.pt_min.data[2*i:2*i+2]
            bbm=res.pt_max.data[2*i:2*i+2]
            bbx=[bbM,bbm]
    print(bbx)
    eY=(bbx[1][1]-215)/215    #experience value
    #print(f'error relative to gripper cam {eY}')


    #eX=(bbx[0][0]-323)/323
    eY=(bbx[1][1]-239)/239
    eX=(bbx[0][0]-312)/312
    #eY=(bbx[1][1]-215)/215    #experience value

    print(f'error relative to gripper cam ex={eX} ey={eY}')
    omni_base.tiny_move(velX=0.5*eX, velY=0.5*-eY, MAX_VEL=0.05) 
print(f'error relative to gripper cam ex={eX} ey={eY}:::::: Error within tolerance')

In [ ]:
bbx

In [ ]:
eY=(bbx[1][1]-239)/239
eX=(bbx[0][0]-312)/312
print(f'error relative to gripper cam ex={eX} ey={eY}')


In [ ]:
#omni_base.tiny_move(velX=0.5*eX, velY=0.5*-eY, MAX_VEL=0.05) 


In [ ]:
clear_octo_client()

av=arm.get_current_joint_values()
print (av,'av')
av[0]+= -0.17###LAB
#av[0]+= -0.35###GAZ
succ=arm.go(av)
rospy.sleep(0.5)
gripper.close(force=0.06)

In [ ]:
av=arm.get_current_joint_values()
print (av,'av')
av[0]+= 0.15
succ=arm.go(av)
omni_base.tiny_move(velX=-0.3, std_time=4.0)
arm.set_named_target('go')
arm.go()
#tf_man.pub_static_tf(point_name='apple')

In [ ]:
plt.imshow(hand_cam.get_image())

In [ ]:
image= cv2.cvtColor(hand_cam.get_image(), cv2.COLOR_RGB2BGR)
img_msg  = bridge.cv2_to_imgmsg(image)
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
debug_image=   cv2.cvtColor(bridge.imgmsg_to_cv2(res.debug_image.image_msgs[0]), cv2.COLOR_RGB2BGR)
plt.imshow(debug_image)

In [ ]:
gripper.open()

In [ ]:
gripper.close(force=0.06)

In [ ]:

for i in range(len(res.poses)):
    tf_man.getTF("head_rgbd_sensor_rgb_frame")
    position = [res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z]
    tf_man.pub_static_tf(pos= position, rot=[0,0,0,1], ref="head_rgbd_sensor_rgb_frame", point_name=res.names[i].data[4:] )   
    rospy.sleep(0.3)
    tf_man.change_ref_frame_tf(res.names[i].data[4:])




In [ ]:
def detect_object_yolo(object_name,res):
    # find object_name in the response message from object_classification service (Yolo)
    objs=[]
    for i,name in enumerate(res.names):
        objs.append(name.data[4:])
        if name.data[4:]==object_name:return res.poses[i]
    if object_name=='all':
        
        return objs
    return []

In [ ]:
#img_msg  = bridge.cv2_to_imgmsg(rgbd.get_image())  #REAL
img_msg  = bridge.cv2_to_imgmsg(cv2.cvtColor(rgbd.get_image(), cv2.COLOR_RGB2BGR ))#GAZEB
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
objects=detect_object_yolo('all',res)   
if len (objects)!=0:
    for i in range(len(res.poses)):
        tf_man.getTF("head_rgbd_sensor_rgb_frame")
        position = [res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z]
        tf_man.pub_static_tf(pos= position, rot=[0,0,0,1], ref="head_rgbd_sensor_rgb_frame", point_name=res.names[i].data[4:] )   
        rospy.sleep(0.3)
        tf_man.change_ref_frame_tf(res.names[i].data[4:])
        rospy.sleep(0.3)
        pose , _=tf_man.getTF(res.names[i].data[4:])
        new_row = {'x': pose[0], 'y': pose[1], 'z': pose[2], 'obj_name': res.names[i].data[4:]}
        objs.loc[len(objs)] = new_row
objs

In [ ]:
objs[['x','y','z']].values

In [ ]:
#get closest object
rob_pos,_=tf_man.getTF('base_link')
np.argmin(np.linalg.norm(rob_pos-objs[['x','y','z']].values  , axis=1))
objs.iloc[np.argmin(np.linalg.norm(rob_pos-objs[['x','y','z']].values  , axis=1))]['obj_name']

In [ ]:
objs[objs['obj_name']==objs.obj_name.values[0]].x.values[0]

# PLANE 1
#### 7.883384	5.948245	0.701945,
#### 8.214565	5.962619	0.700034,
#### 8.213697	6.107727	0.709487,
####  7.893623	6.082935	0.693285

# PLANE 2
#### 7.88076	6.086619	0.416096,
#### 8.214128	5.9663	0.416672

# Plane 3
#### 8.20238	5.963262	0.117541

In [ ]:
head.set_named_target('neutral')

av=arm.get_current_joint_values()
av[0]=0.5
av[1]=-0.5
arm.go(av)
head.set_joint_values([-np.pi/2 , -0.5])

In [ ]:
head.set_joint_values([-np.pi/2 , -0.5])

In [ ]:


av=arm.get_current_joint_values()
av[0]=0.05

arm.go(av)

In [ ]:
head.set_joint_values([-np.pi/2 , -0.8])

av=arm.get_current_joint_values()
av[0]=0.0
av[1]=-0.5
arm.go(av)

In [2]:
import pandas as pd


[ WARN] [1717034039.547825191]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 165.754000 according to authority /pose_integrator
[ WARN] [1717034039.547886545]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 165.754000 according to authority /pose_integrator
[ WARN] [1717034039.547918453]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 165.754000 according to authority /pose_integrator
[ WARN] [1717034039.547963091]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 165.754000 according to authority /pose_integrator
[ WARN] [1717034039.547993025]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 165.754000 according to authority /pose_integrator
[ WARN] [1717034039.548028747]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 165.754000 according to authority /pose_integrator
[ WARN] [1717034040.38051578

[ WARN] [1717034089.833395485]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 179.963000 according to authority /pose_integrator
[ WARN] [1717034089.833451982]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 179.963000 according to authority /pose_integrator
[ WARN] [1717034089.833489464]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 179.963000 according to authority /pose_integrator
[ WARN] [1717034089.833526418]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 179.963000 according to authority /pose_integrator
[ WARN] [1717034089.833555103]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 179.963000 according to authority /pose_integrator
[ WARN] [1717034089.833587013]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 179.963000 according to authority /pose_integrator
[ WARN] [1717034090.51202363

[ WARN] [1717034113.075212941]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 186.675000 according to authority /pose_integrator
[ WARN] [1717034113.075259679]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 186.675000 according to authority /pose_integrator
[ WARN] [1717034113.075286545]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 186.675000 according to authority /pose_integrator
[ WARN] [1717034113.075310986]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 186.675000 according to authority /pose_integrator
[ WARN] [1717034113.075347758]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 186.675000 according to authority /pose_integrator
[ WARN] [1717034113.075374249]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 186.675000 according to authority /pose_integrator
[ WARN] [1717034121.67559956

In [ ]:
import pandas as pd
objs = pd.read_csv ('/home/roboworks/Documents/objs.csv')
objs=objs.drop(columns='Unnamed: 0')

In [ ]:
objs=objs[objs['obj_name'] != 'bowl']

In [ ]:
objs

In [ ]:
objs[objs['obj_name'] != 'bowl']

In [ ]:
objs

In [ ]:
from sklearn.cluster import KMeans
shelf_objs=objs[objs['shelves']==True ]
sh_o_btm=shelf_objs[shelf_objs['z']<0.2]


In [ ]:
'other' in sh_o_top['category'].values

In [ ]:
shelf_objs=objs[objs['shelves']==True ]

sh_o_btm=shelf_objs[shelf_objs['z']<0.2]
sh_o_mdl=shelf_objs[(shelf_objs['z']>0.2 )& (shelf_objs['z']<0.5) ]
sh_o_top=shelf_objs[(shelf_objs['z']>0.5 )& (shelf_objs['z']<1.5) ]
sh_o_btm['category'].value_counts(),sh_o_mdl['category'].value_counts() 

a= sh_o_btm['category'].value_counts()
if 'other' in sh_o_btm['category'].values:a.drop('other', inplace=True)
print(f'btm shelf category {a.index[a.argmax()]}')

a= sh_o_mdl['category'].value_counts()
if 'other' in sh_o_mdl['category'].values:a.drop('other', inplace=True)
print(f'middle shelf category {a.index[a.argmax()]}')


a= sh_o_top['category'].value_counts()
if 'other' in sh_o_top['category'].values:a.drop('other', inplace=True)
print(f'top shelf category {a.index[a.argmax()]}')


In [ ]:
a

In [ ]:
kitchen =['bowl','spatula','spoon', 'bowl']
tools=['extra_large_clamp','large_clamp','small_clamp','medium_clamp','adjustable_wrench',
'flat_screwdriver','phillips_screwdriver','wood_block']
balls= ['softball','tennis_ball']
fruits= ['apple','banana', 'lemon','pear']
food =['chips_can'
,'mustard_bottle'
,'potted_meat_can'
,'tomato_soup_can'
,'tuna_fish_can']

def categorize_objs(name):
    if name in kitchen: return 'kitchen'
    elif name in tools: return 'tools'
    elif name in balls: return 'balls'
    elif name in fruits: return 'fruits'
    elif name in food: return 'food'
    return 'other'
cats=[]
for name in objs['obj_name']:
    cats.append(categorize_objs(name))
objs['category'] = cats   


In [ ]:
def categorize_objs(name):
    if name in kitchen: return 'kitchen'
    elif name in tools: return 'tools'
    elif name in balls: return 'balls'
    elif name in fruits: return 'fruits'
    elif name in food: return 'food'
    return 'other'
    
    

In [ ]:
cats=[]
for name in objs['obj_name']:
    cats.append(categorize_objs(name))
objs['category'] = cats

In [ ]:
objs

In [ ]:
import numpy as np
regions={'shelves':np.load('/home/roboworks/Documents/shelf_sim.npy'),'pickup':np.load('/home/roboworks/Documents/pickup_sim.npy')}
def is_inside(x,y):return ((area_box[:,1].max() > y) and (area_box[:,1].min() < y)) and ((area_box[:,0].max() > x) and (area_box[0,0].min() < x)) 
for name in regions:
    in_region=[]
    area_box=regions[name]
    print (area_box,name)
    for index, row in objs[['x','y','z']].iterrows():
        in_region.append(is_inside(row.x, row.y))
        print (row.x, row.y,is_inside(row.x, row.y))
    objs[name]=pd.Series(in_region)
objs

In [ ]:
(area_box[:,1].max() > row.y),(area_box[:,1].min() < row.y),((area_box[:,0].max() > row.x) , (area_box[0,0].min() < row.x)) 

In [ ]:
def is_inside(x,y):return ((area_box[:,1].max() > y) and (area_box[:,1].min() < y)) and ((area_box[:,0].max() > x) and (area_box[0,0].min() < x)) 

In [ ]:
pickup_region=np.zeros((2,2))
#np.save('/home/roboworks/Documents/pickup_region.npy', pickup_region)

In [ ]:
ab[0,0],ab[0,1]=6,6
ab[1,0],ab[1,1]=7,4.0

In [ ]:
import numpy as np

#ab=np.load('/home/roboworks/Documents/pickup_region.npy')
ab= np.load('/home/roboworks/Documents/pickup_sim.npy')

In [ ]:

ab[0,0]=5.8
ab

In [ ]:
np.save('/home/roboworks/Documents/pickup_region.npy',ab)


In [ ]:
#np.save('/home/roboworks/Documents/shelf_sim.npy',ab)
np.save('/home/roboworks/Documents/pickup_sim.npy',ab)

In [ ]:
objs[objs['pickup']==True][['x','y','z']].values

In [ ]:
rob_pos,_=tf_man.getTF('base_link')
ix=np.argmin(np.linalg.norm(rob_pos-objs[objs['pickup']==True][['x','y','z']].values  , axis=1))
ix


In [ ]:
rob_pos,_=tf_man.getTF('base_link')
ix=np.argmin(np.linalg.norm(rob_pos-objs[objs['pickup']==True][['x','y','z']].values  , axis=1))
objs[objs['pickup']==True][['obj_name','category']].iloc[ix]


In [ ]:
ix=np.argmin(np.linalg.norm(rob_pos-objs[objs['pickup']==True][['x','y','z']].values  , axis=1))

In [ ]:
pose=np.asarray((8.212561, 6.028789, 0.12))
pose

In [ ]:
tf_man.pub_static_tf(pos=pose,point_name='PLACING')

In [ ]:
#pose=np.asarray((8.212561, 6.028789, 0.12))
pose=np.asarray((8.062455, 6.0622816, 0.6610926))
tf_man.pub_static_tf(pos=pose,point_name='PLACING')

In [ ]:
arm['arm_lift_joint']=0.0

In [ ]:
omni_base.move_base(known_location='shelf')

In [ ]:
omni_base.tiny_move(velT=-0.7,std_time=1)

In [ ]:
av=arm.get_current_joint_values()
high_shelf_place=av
high_shelf_place


In [ ]:
place_high_shelf=np.asarray([0.15050079265248406,
 -0.5861087653247212,
 0.0850017715788356,
 -0.9342371117035135,
 0.022035143687695324,
 0.0])

In [ ]:
arm.go(place_high_shelf)

av=arm.get_current_joint_values()


In [ ]:
av=arm.get_current_joint_values()
high_shelf_place=av

In [ ]:
arm.go(high_shelf_place)

In [ ]:
av=arm.get_current_joint_values()
mid_shelf_place=av
mid_shelf_place

In [ ]:
av=arm.get_current_joint_values()
low_shelf_place=av
low_shelf_place

In [ ]:
high_shelf_place,low_shelf_place,mid_shelf_place

In [ ]:
high_shelf_place=np.asarray([0.4337121304669095,
 -1.3139266720690426,
 0.08621383272112926,
 -0.4260850221072081,
 0.02285051353933021,
 0.0])

In [ ]:
low_shelf_place=np.asarray([-5.465250513158978e-07,
 -1.8875162206087301,
 0.08646571240675005,
 0.3040738848075222,
 0.022992974203400962,
 0.0]
)

In [ ]:
high_shelf_place=np.asarray([0.15050079265248406,
 -0.5861087653247212,
 0.0850017715788356,
 -0.9342371117035135,
 0.022035143687695324,
 0.0])
mid_shelf_place=np.asarray([0.2556342351169577,
 -1.6040815280188854,
 -0.008078345343693893,
 -0.05795663028496589,
 0.0019971082238159354,
 0.0])

low_shelf_place=np.asarray([0.04577467546491997,
 -2.262541528018885,
 0.00010865465630605087,
 0.701618369715034,
 -0.0003068917761841483,
 0.0]
)

In [ ]:
tf_man.getTF('placing_area_top_shelf1')

In [ ]:
request= segmentation_server.request_class() 
#r = cv2.getTrackbarPos('Plane height (cms)', 'class rgbd')
#request.height.data=0.9
print ('#############Finding placing in plane####################',request.height.data)
request.height.data=-1
#head.set_joint_values([ 0.1, -0.5])
res=placing_finder_server.call(request)
#succ=seg_res_tf(res)
print (f'Placing Area at {res.poses.data}')
tf_man.pub_static_tf(pos=[res.poses.data[0], res.poses.data[1],res.poses.data[2]], rot =[0,0,0,1], point_name='placing_area')
img=bridge.imgmsg_to_cv2(res.im_out.image_msgs[0])
#cv2.imshow('our of res'  , img)
plt.imshow(img)

In [ ]:
tf_man.getTF('placing_area')

In [ ]:
head.set_named_target('neutral')

In [ ]:
arm.set_named_target('go')
arm.go()

In [ ]:
res = omni_base.move_base(known_location='place_shelf', time_out=10)

In [ ]:
high_shelf_place=[0.65,         -1.139,         0.08621,        -0.4260,        0.02285,         0.0]#REAL
high_shelf_place=[0.4337,         -1.3139,         0.08621,        -0.4260,        0.02285,         0.0] #GAZ

In [ ]:
arm.set_joint_value_target(high_shelf_place)
arm.go()

In [ ]:
def base_grasp_D(tf_name,d_x=0.66,d_y=-0.1,timeout=1.0):
    timeout = rospy.Time.now().to_sec() + timeout
    rob_pos,rot=tf_man.getTF('base_link')    
    original_rot=tf.transformations.euler_from_quaternion(rot)[2]
    succ = False 
    target_object= tf_name        
    while (timeout >= rospy.Time.now().to_sec()) and not succ:            
        _,rot= tf_man.getTF("base_link",ref_frame='map')
        trans,_=tf_man.getTF(target_object,ref_frame="base_link")
        #trans
        eX, eY, eZ = trans
        eX+= -d_x  #x offest
        eY+= -d_y #y Offset
        eT= tf.transformations.euler_from_quaternion(rot)[2] - original_rot #Original 
        print (eT)
        if eT > np.pi: eT=-2*np.pi+eT
        if eT < -np.pi: eT= 2*np.pi+eT
        rospy.loginfo("error: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(eX, eY , eT,target_object))
        X, Y, Z = trans
        rospy.loginfo("Pose: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(X, Y , eT,target_object))
        if abs(eX) <=0.05 :
            print ('here')
            eX = 0
        if abs(eY) <=0.05  :
            eY = 0
        if abs(eT   ) < 0.1:
            eT = 0
        succ =  eX == 0 and eY == 0 and eT==0            
        omni_base.tiny_move( velX=0.2*+eX,velY=0.3*eY, velT=-eT,std_time=0.2, MAX_VEL=0.3) 

    

In [ ]:
tf_man.getTF('placing_area')

In [ ]:
base_grasp_D(tf_name='placing_area')

In [ ]:
base_grasp_D(tf_name='placing_area',d_x=0.76, d_y=0.0,timeout=30)

In [ ]:
gripper.open()

In [ ]:
target_object='placing_area'
_,rot= tf_man.getTF("base_link",ref_frame='map')
trans,_=tf_man.getTF(target_object,ref_frame="base_link")

 
eX, eY, eZ = trans
eT= tf.transformations.euler_from_quaternion(rot)[2] - original_rot #Original 
print (eT)
if eT > np.pi: eT=-2*np.pi+eT
if eT < -np.pi: eT= 2*np.pi+eT
rospy.loginfo("error: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(eX, eY , eT,target_object))
X, Y, Z = trans
rospy.loginfo("Pose: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(X, Y , eT,target_object))

In [ ]:
omni_base.tiny_move( velX=0.1*+eX,velY=0.1*eY, velT=-eT,std_time=0.2, MAX_VEL=0.3) 

In [ ]:
d_x=0.58
d_y= 0.1
_,rot= tf_man.getTF("base_link",ref_frame='map')
trans,_=tf_man.getTF(target_object,ref_frame="base_link")
#trans
eX, eY, eZ = trans
eX+= -d_x  #x offest
eY+= -d_y #y Offset
eT= tf.transformations.euler_from_quaternion(rot)[2] - original_rot #Original 
print (eT)
if eT > np.pi: eT=-2*np.pi+eT
if eT < -np.pi: eT= 2*np.pi+eT
rospy.loginfo("error: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(eX, eY , eT,target_object))
X, Y, Z = trans
rospy.loginfo("Pose: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(X, Y , eT,target_object))
if abs(eX) <=0.05 :
    print ('here')
    eX = 0
if abs(eY) <=0.05  :
    eY = 0
if abs(eT   ) < 0.1:
    eT = 0
succ =  eX == 0 and eY == 0 and eT==0      
succ
#omni_base.tiny_move( velX=0.2*+eX,velY=0.3*eY, velT=-eT,std_time=0.2, MAX_VEL=0.3) 


In [ ]:
omni_base.tiny_move( velX=0.1*+eX,velY=0.1*eY, velT=-eT,std_time=0.2, MAX_VEL=0.3) 

In [ ]:
arm.get_current_pose()

In [ ]:
av=arm.get_current_joint_values()
av[0]+=-0.1
arm.go(av)

In [ ]:
arm.set_named_target('go')
arm.go()

In [ ]:
omni_base.move_base('place_shelf')

In [ ]:
whole_body.allow_replanning(True)
whole_body.set_num_planning_attempts(10)
whole_body.set_planning_time(10.0)
whole_body.set_workspace([-2.0, -2.0, 0.0, 2.0, 2.0, 2.0])

In [ ]:
object_point = PointStamped()
object_point

In [ ]:
target_position=[1.0,1.0,1.0]
object_point = PointStamped()
object_point.header.frame_id = "base_link"
object_point.point.x = target_position[0]
object_point.point.y = target_position[1]
object_point.point.z = target_position[2]
#transformar la posicion del objeto al marco de referencia de la base del robot

try:
    transformed_object_point = tfBuffer.transform(object_point, "base_link", timeout=rospy.Duration(1))
    transformed_base = self.tfBuffer.lookup_transform("odom", "base_link", rospy.Time(0), timeout=rospy.Duration(1))
except :
    rospy.WARN("Error al transformar la posicion del objeto al marco de referencia")


In [ ]:
tfBuffer.transform(object_point,'base_link',timeout=rospy.Duration(3))

In [ ]:
whole_body.set_start_state_to_current_state()
whole_body.set_position_target(position_goal)
succ = whole_body.go(wait= True)
rospy.sleep(0.5)
whole_body.stop()

In [ ]:
tfBuffer = tf2_ros.Buffer()


In [ ]:
omni_base.move_base(known_location='pickup', time_out=200)

In [ ]:
head.set_joint_values([ 0.0, -0.5])


In [ ]:
img_msg  = bridge.cv2_to_imgmsg( cv2.cvtColor(rgbd.get_image(), cv2.COLOR_RGB2BGR))### GAZEBO BGR!?!??!
req      = classify_client.request_class()
req.in_.image_msgs.append(img_msg)
res      = classify_client(req)
objects=detect_object_yolo('all',res)   

In [ ]:
rospack = rospkg.RosPack()
file_path = rospack.get_path('config_files') 
objs = pd.read_csv (file_path+'/objects.csv')
objs=objs.drop(columns='Unnamed: 0')
print (objs)

In [ ]:
def check_if_grasped(pose_target,test_pt,tolerance=0.05):    
    return np.linalg.norm(pose_target-obj_pt)<tolerance

In [ ]:
np.linalg.norm(pose_target-obj_pt)

In [ ]:
test_pt=objs[objs['obj_name']=='orange'][['x','y','z']].values
check_if_grasped(pose_target,test_pt)

In [ ]:
def check_if_grasped(pose_target,test_pt,tolerance=0.05):return np.linalg.norm(pose_target-test_pt)<tolerance


##############################
pose_target,_=tf_man.getTF('plum')
#########################

if len (objects)!=0 :
    for i in range(len(res.poses)):
        
        position = [res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z]
        
        object_point = PointStamped()
        object_point.header.frame_id = "head_rgbd_sensor_rgb_frame"
        object_point.point.x = position[0]
        object_point.point.y = position[1]
        object_point.point.z = position[2]
        position_map = tfBuffer.transform(object_point, "map", timeout=rospy.Duration(1))
        
        tf_man.pub_static_tf(pos= [position_map.point.x,position_map.point.y,position_map.point.z], rot=[0,0,0,1], ref="map", point_name=res.names[i].data[4:] )
        test_pt=np.asarray((position_map.point.x,position_map.point.y,position_map.point.z))
        print (np.linalg.norm(pose_target-test_pt))
        if check_if_grasped(pose_target,test_pt):
            print (f'Centroid found in area {test_pt}, obj_name: {res.names[i].data[4:]}')
            print ('Grasping May have failed')
        
        
        
        
        

In [ ]:

if len (objects)!=0 :
    for i in range(len(res.poses)):
        #tf_man.getTF("head_rgbd_sensor_rgb_frame")
        position = [res.poses[i].position.x ,res.poses[i].position.y,res.poses[i].position.z]
        print ('position,name',position,res.names[i].data[4:])
        ##########################################################

        object_point = PointStamped()
        object_point.header.frame_id = "head_rgbd_sensor_rgb_frame"
        object_point.point.x = position[0]
        object_point.point.y = position[1]
        object_point.point.z = position[2]
        position_map = tfBuffer.transform(object_point, "map", timeout=rospy.Duration(1))
        
        
        
        print ('position_map',position_map)
        tf_man.pub_static_tf(pos= [position_map.point.x,position_map.point.y,position_map.point.z], rot=[0,0,0,1], ref="map", point_name=res.names[i].data[4:] )
        new_row = {'x': position_map.point.x, 'y': position_map.point.y, 'z': position_map.point.z, 'obj_name': res.names[i].data[4:]}
        objs.loc[len(objs)] = new_row
        ###########################################################

In [ ]:
head.to_tf('plum')

In [ ]:
base_grasp_D('plum',timeout=30)

In [ ]:
objs['x']

In [ ]:
obj_pt=np.asarray((position_map.point.x,
position_map.point.y,
position_map.point.z))

if np.linalg.norm(pose_target-obj_pt)<0.05:print ('I say chale man')

In [ ]:
pose_target,_=tf_man.getTF('plum')

In [ ]:
objs['x']

In [ ]:

np.linalg.norm(pose_target-obj_pt)

In [ ]:
pose_target

In [ ]:
obj_pt

In [ ]:
av=arm.get_current_joint_values()
av

In [ ]:
brazo.set_joint_values(av[:-1])

In [ ]:
av= arm.get_current_joint_values()

In [ ]:
av

In [ ]:
head.set_joint_values([0.0 , 0.0])
av=arm.get_current_joint_values()
av[0]=0.67
av[1]=-0.74
arm.go(av)
head.set_joint_values([-np.pi/2 , -0.7])     

In [ ]:
rospack = rospkg.RosPack()        
file_path = rospack.get_path('config_files') 
objs = pd.read_csv (file_path+'/objects.csv') #EMPTY DATAFRAME
objs=objs.drop(columns='Unnamed: 0')
print (objs)
file_path = rospack.get_path('config_files')+'/regions'         
regions={'shelves':np.load(file_path+'/shelves_region.npy'),'pickup':np.load(file_path+'/pickup_region.npy')}   ## KNOWN REGIONS
print (f'Regions for Storing Groceries(Real Robot) {regions}')
##TO AVOID SMACH DYING IN CASE NO PLACING AREA IS FOUND, THere is a default that at least allows the test to continue
x,y= np.mean(regions['shelves'], axis=0)
z=0.44#self.mid_shelf_height=0.4 shelves heights must also be set on SCAN SHELF  state init section.

In [ ]:
tf_man.pub_static_tf(pos=[x,y,0.89],point_name='top_shelf')
tf_man.pub_static_tf(pos=[x,y,0.42],point_name='mid_shelf')
tf_man.pub_static_tf(pos=[x,y,0.05],point_name='low_shelf')
#head.to_tf()

In [ ]:
head.to_tf('low_shelf')

In [ ]:
head.set_joint_values([-0.3,-0.47])

In [ ]:
d_x=0.66
d_y=-0.1
timeout=1.0
rob_pos,rot=tf_man.getTF('base_link')    
original_rot=tf.transformations.euler_from_quaternion(rot)[2]
succ = False 
target_object= 'placing_area'        

In [ ]:

i=0
while  not succ and not rospy.is_shutdown():
    i+=1
    _,rot= tf_man.getTF("base_link",ref_frame='map')
    trans,_=tf_man.getTF(target_object,ref_frame="base_link")
    #trans
    eX, eY, eZ = trans
    eX+= -d_x  #x offest
    eY+= -d_y #y Offset
    eT= tf.transformations.euler_from_quaternion(rot)[2] - original_rot #Original 

    rospy.loginfo("error: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(eX, eY , eT,target_object))
    #if eT > np.pi: eT=-2*np.pi+eT    #  angle error constricted to values between pi and -pi  
    #if eT < -np.pi: eT= 2*np.pi+eT   
    eT = (eT + np.pi) % (2 * np.pi) - np.pi

    X, Y, Z = trans
    rospy.loginfo("Pose: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(X, Y , eT,target_object))
    if abs(eX) <=0.05 :
        print ('here')
        eX = 0
    if abs(eY) <=0.05  :
        eY = 0
    if abs(eT   ) < 0.03:
        eT = 0
    succ =  eX == 0 and eY == 0 and eT==0         
    velX=0.2*eX
    velY=0.3*eY
    ############################################
    if   velX <= 0: corr_velX= min (-0.0051,velX)
    elif velX >  0: corr_velX= max ( 0.0051,velX)
    if   velY <= 0: corr_velY= min (-0.0051,velY)
    elif velY >  0: corr_velY= max ( 0.005,velY)
    #############################################
    if i %3 ==0 :
        print("Pose: {:.2f}, {:.2f}, angle {:.2f}, target obj frame {}".format(X, Y , eT,target_object))
        i=0
    omni_base.tiny_move( velX=corr_velX,velY=corr_velY, velT=-eT,std_time=0.2, MAX_VEL=0.3) 

    

In [ ]:
omni_base.tiny_move( velX=corr_velX,velY=corr_velY, velT=-eT,std_time=0.2, MAX_VEL=0.5, MAX_VEL_THETA=0.3) 

In [ ]:

succ=False
THRESHOLD = 0.03
_, rot  = tf_man.getTF("base_link",ref_frame='map')
original_rot=tf.transformations.euler_from_quaternion(rot)[2]
    

In [ ]:

while  not succ and not rospy.is_shutdown():
    trans,_ = tf_man.getTF(target_frame='placing_area', ref_frame='hand_palm_link')
    _, rot  = tf_man.getTF("base_link",ref_frame='map')

    if type(trans) is not bool:
        _, eY, eX = trans

    if abs(eY) < THRESHOLD:
        eY = 0
    if abs(eX) < THRESHOLD:
        eX = 0
    eT= tf.transformations.euler_from_quaternion(rot)[2] - original_rot #Original 

    eT = (eT + np.pi) % (2 * np.pi) - np.pi

    print("error: {:.2f}, {:.2f}, angle {:.2f}, target obj frame placing area".format(eX, eY , eT))
    if eT > np.pi: eT=-2*np.pi+eT
    if eT < -np.pi: eT= 2*np.pi+eT
    if abs(eT) < 0.05:
        eT=0

    if eX >0: velX = max( 0.005,eX)
    if eX <=0: velX = min(-0.005,eX)
    if eY >0: velY = max( 0.005,eY)
    if eY <=0: velY = min(-0.005,eY)
    print("error: {:.2f}, {:.2f}, angle {:.2f}, target obj frame placing area".format(eX, eY , eT))
    succ =  eX == 0 and eY == 0 and eT==0            
    omni_base.tiny_move( velX=0.1*velX,velY=-0.4*velY, velT=-eT,std_time=0.2, MAX_VEL=0.1, MAX_VEL_THETA=0.1) 


In [ ]:
omni_base.tiny_move( velX=0.1*velX,velY=-0.4*velY, velT=-eT,std_time=0.2, MAX_VEL=0.1, MAX_VEL_THETA=0.1) 


In [ ]:
#omni_base.tiny_move( velX=-1,std_time=1.5,MAX_VEL=1) 

In [ ]:
av=arm.get_current_joint_values()
av

In [ ]:
av=arm.get_current_joint_values()
av

In [ ]:
 transformed_object_point = self.tf2_buffer.transform(object_point, "odom", timeout=rospy.Duration(1))

In [2]:
tf_man.pub_static_tf(pos=[4.96,1.41,0],rot=[0,0,0.707,0.707],point_name='pikcup')

In [3]:
pose, quat=tf_man.getTF('pikcup')

In [8]:
offset_point=[0.3,0.0,0.01]          # Offset relative to object tf
        

In [9]:
object_point = PointStamped()
object_point.header.frame_id = "pikcup"#"base_link"
object_point.point.x = offset_point[0]
object_point.point.y = offset_point[1]
object_point.point.z = offset_point[2]



In [10]:

transformed_object_point = tfBuffer.transform(object_point, "map", timeout=rospy.Duration(1))

tf_man.pub_static_tf(pos=[transformed_object_point.point.x,transformed_object_point.point.y,transformed_object_point.point.z],rot=quat,point_name='pikcup+2')

[ WARN] [1716695051.232967007]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 663.496000 according to authority /pose_integrator
[ WARN] [1716695051.233016625]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 663.496000 according to authority /pose_integrator
[ WARN] [1716695051.233055029]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 663.496000 according to authority /pose_integrator
[ WARN] [1716695051.233086759]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 663.496000 according to authority /pose_integrator
[ WARN] [1716695051.233128328]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 663.496000 according to authority /pose_integrator
[ WARN] [1716695051.233167321]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 663.496000 according to authority /pose_integrator
[ WARN] [1716695252.00705165

[ WARN] [1716695383.571807386]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 751.179000 according to authority /pose_integrator
[ WARN] [1716695383.571857394]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 751.179000 according to authority /pose_integrator
[ WARN] [1716695383.571889301]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 751.179000 according to authority /pose_integrator
[ WARN] [1716695383.571919434]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 751.179000 according to authority /pose_integrator
[ WARN] [1716695383.571951073]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 751.179000 according to authority /pose_integrator
[ WARN] [1716695383.571988048]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 751.179000 according to authority /pose_integrator
[ WARN] [1716695393.34849503

shutdown request: [/smach] Reason: new node registered with same name


In [ ]:
static_transform_stamped = TransformStamped()
quat=[0,0,0.707,0.707]
# Fill the TransformStamped message with data
static_transform_stamped.header.stamp = rospy.Time(0)
static_transform_stamped.header.frame_id = 'map'  # parent frame
static_transform_stamped.child_frame_id = 'pikcup+2'   # child frame

# Translation (x, y, z)
static_transform_stamped.transform.translation.x = transformed_object_point.point.x
static_transform_stamped.transform.translation.y = transformed_object_point.point.y
static_transform_stamped.transform.translation.z = transformed_object_point.point.z

# Rotation (roll, pitch, yaw)

static_transform_stamped.transform.rotation.x = quat[0]
static_transform_stamped.transform.rotation.y = quat[1]
static_transform_stamped.transform.rotation.z = quat[2]
static_transform_stamped.transform.rotation.w = quat[3]

# Broadcast the static transform
broad.sendTransform(static_transform_stamped)


In [ ]:

lis= tf2_ros.TransformListener(tfBuffer)
broad= tf2_ros.StaticTransformBroadcaster()
#broad.pub_tf()

In [ ]:
static_transform_stamped = TransformStamped()
quat=[0,0,0.707,0.707]
# Fill the TransformStamped message with data
static_transform_stamped.header.stamp = rospy.Time(0)
static_transform_stamped.header.frame_id = 'map'  # parent frame
static_transform_stamped.child_frame_id = 'pikcup'   # child frame

# Translation (x, y, z)
static_transform_stamped.transform.translation.x = pose[0]
static_transform_stamped.transform.translation.y = pose[1]
static_transform_stamped.transform.translation.z = pose[2]

# Rotation (roll, pitch, yaw)

static_transform_stamped.transform.rotation.x = quat[0]
static_transform_stamped.transform.rotation.y = quat[1]
static_transform_stamped.transform.rotation.z = quat[2]
static_transform_stamped.transform.rotation.w = quat[3]

# Broadcast the static transform
broad.sendTransform(static_transform_stamped)


In [ ]:

tf_man.pub_static_tf(pos=translated_point,quat=[0,0,0.707,0.707],point_name='pikcup+1')

In [36]:
o=read_yaml('/regions/regions_sim.yaml')
df=pd.DataFrame.from_dict(o)
area_bo_x=df['shelves'][['x_min','x_max']].values
area_bo_y=df['shelves'][['y_min','y_max']].values
area_box=np.concatenate((area_bo_x,area_bo_y)).reshape((2,2)).T

[ WARN] [1717041863.249153510]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2144.961000 according to authority /pose_integrator
[ WARN] [1717041863.249217864]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2144.961000 according to authority /pose_integrator
[ WARN] [1717041863.249268056]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2144.961000 according to authority /pose_integrator
[ WARN] [1717041863.249311775]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2144.961000 according to authority /pose_integrator
[ WARN] [1717041863.249358540]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2144.961000 according to authority /pose_integrator
[ WARN] [1717041863.249406388]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2144.961000 according to authority /pose_integrator


In [37]:
string_list=df['shelf']['z'].split(',')
top,mid,low=[list(map(float, s.split(','))) for s in string_list]

KeyError: 'shelf'

In [39]:
area_bo_x=df['shelves'][['x_min','x_max']].values
area_bo_y=df['shelves'][['y_min','y_max']].values
area_box=np.concatenate((area_bo_x,area_bo_y)).reshape((2,2)).T
print (area_box)
print (area_box[:,1])

#def is_inside(x,y,z):    return ((area_box[:,1].max() > y) and (area_box[:,1].min() < y)) and ((area_box[:,0].max() > x) and (area_box[0,0].min() < x)) and (pickup_plane_z<z)  
def is_inside_top(x,y,z):return ((area_box[:,1].max() > y) and (area_box[:,1].min() < y)) and ((area_box[:,0].max() > x) and (area_box[0,0].min() < x))    and (top_shelf_height < z )  
def is_inside_mid(x,y,z):return ((area_box[:,1].max() > y) and (area_box[:,1].min() < y)) and ((area_box[:,0].max() > x) and (area_box[0,0].min() < x))    and ((0.9*top_shelf_height > z) and (mid_shelf_height < z  )  )  
def is_inside_low(x,y,z):return ((area_box[:,1].max() > y) and (area_box[:,1].min() < y)) and ((area_box[:,0].max() > x) and (area_box[0,0].min() < x))    and ((0.9*mid_shelf_height > z  )  )

[[3.08 1.3]
 [4.05 2.0]]
[1.3 2.0]


[ WARN] [1717041900.893025378]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2154.151000 according to authority /pose_integrator
[ WARN] [1717041900.893078167]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2154.151000 according to authority /pose_integrator
[ WARN] [1717041900.893104310]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2154.151000 according to authority /pose_integrator
[ WARN] [1717041900.893129002]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2154.151000 according to authority /pose_integrator
[ WARN] [1717041900.893156931]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2154.151000 according to authority /pose_integrator
[ WARN] [1717041900.893183741]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2154.151000 according to authority /pose_integrator
[ WARN] [1717041901.73

In [40]:
is_inside_low(1,2,3)

False

[ WARN] [1717041942.787513563]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2164.492000 according to authority /pose_integrator
[ WARN] [1717041942.787571020]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2164.492000 according to authority /pose_integrator
[ WARN] [1717041942.787611261]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2164.492000 according to authority /pose_integrator
[ WARN] [1717041942.787644625]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2164.492000 according to authority /pose_integrator
[ WARN] [1717041942.787680697]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2164.492000 according to authority /pose_integrator
[ WARN] [1717041942.787719825]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2164.492000 according to authority /pose_integrator
[ WARN] [1717041947.55

[ WARN] [1717042048.078600327]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2190.976000 according to authority /pose_integrator
[ WARN] [1717042048.078648768]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2190.976000 according to authority /pose_integrator
[ WARN] [1717042048.078681889]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2190.976000 according to authority /pose_integrator
[ WARN] [1717042048.078709898]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2190.976000 according to authority /pose_integrator
[ WARN] [1717042048.078742520]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2190.976000 according to authority /pose_integrator
[ WARN] [1717042048.078771909]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2190.976000 according to authority /pose_integrator
[ WARN] [1717042080.68

[ WARN] [1717042176.696663583]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2222.469000 according to authority /pose_integrator
[ WARN] [1717042176.696723107]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2222.469000 according to authority /pose_integrator
[ WARN] [1717042176.696760861]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2222.469000 according to authority /pose_integrator
[ WARN] [1717042176.696804344]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2222.469000 according to authority /pose_integrator
[ WARN] [1717042176.696858193]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2222.469000 according to authority /pose_integrator
[ WARN] [1717042176.696905933]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2222.469000 according to authority /pose_integrator
[ WARN] [1717042194.75

[ WARN] [1717042394.225590517]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.491000 according to authority /pose_integrator
[ WARN] [1717042394.225658674]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.491000 according to authority /pose_integrator
[ WARN] [1717042394.225706219]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.491000 according to authority /pose_integrator
[ WARN] [1717042394.225748039]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.491000 according to authority /pose_integrator
[ WARN] [1717042394.225790117]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.491000 according to authority /pose_integrator
[ WARN] [1717042394.225854216]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2275.491000 according to authority /pose_integrator
[ WARN] [1717042426.60

...

[ WARN] [1717043289.027497126]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2503.463000 according to authority /pose_integrator
[ WARN] [1717043289.027609078]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2503.463000 according to authority /pose_integrator
[ WARN] [1717043289.027643791]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2503.463000 according to authority /pose_integrator
[ WARN] [1717043289.027680747]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2503.463000 according to authority /pose_integrator
[ WARN] [1717043289.027713687]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2503.463000 according to authority /pose_integrator
[ WARN] [1717043289.027761503]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2503.463000 according to authority /pose_integrator
[ WARN] [1717043297.30

[ WARN] [1717043432.980965492]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2537.512000 according to authority /pose_integrator
[ WARN] [1717043432.981047200]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2537.512000 according to authority /pose_integrator
[ WARN] [1717043432.981093879]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2537.512000 according to authority /pose_integrator
[ WARN] [1717043432.981141368]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2537.512000 according to authority /pose_integrator
[ WARN] [1717043432.981187108]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2537.512000 according to authority /pose_integrator
[ WARN] [1717043432.981233228]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2537.512000 according to authority /pose_integrator
[ WARN] [1717043437.50

[ WARN] [1717043513.221429263]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2557.857000 according to authority /pose_integrator
[ WARN] [1717043513.221475444]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2557.857000 according to authority /pose_integrator
[ WARN] [1717043513.221514901]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2557.857000 according to authority /pose_integrator
[ WARN] [1717043513.221556572]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2557.857000 according to authority /pose_integrator
[ WARN] [1717043513.221589654]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2557.857000 according to authority /pose_integrator
[ WARN] [1717043513.221624028]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2557.857000 according to authority /pose_integrator
[ WARN] [1717043543.16

[ WARN] [1717043718.654538810]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2610.353000 according to authority /pose_integrator
[ WARN] [1717043718.654583279]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2610.353000 according to authority /pose_integrator
[ WARN] [1717043718.654616194]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2610.353000 according to authority /pose_integrator
[ WARN] [1717043718.654649299]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2610.353000 according to authority /pose_integrator
[ WARN] [1717043718.654686941]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2610.353000 according to authority /pose_integrator
[ WARN] [1717043718.654719137]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2610.353000 according to authority /pose_integrator
[ WARN] [1717043746.21

[ WARN] [1717043965.079795426]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2668.872000 according to authority /pose_integrator
[ WARN] [1717043965.079844291]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2668.872000 according to authority /pose_integrator
[ WARN] [1717043965.079875448]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2668.872000 according to authority /pose_integrator
[ WARN] [1717043965.079904420]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2668.872000 according to authority /pose_integrator
[ WARN] [1717043965.079936129]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2668.872000 according to authority /pose_integrator
[ WARN] [1717043965.079992188]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 2668.872000 according to authority /pose_integrator


In [5]:
df


,pickup,shelf
x_min,-1.23,3.08
x_max,-0.77,4.05
y_min,0.41,1.3
y_max,0.63,2.0
z,0.41,"0.08,0.41,0.01"


In [6]:
np.concatenate((area_box,area_boy)).reshape((2,2)).T

NameError: name 'area_boy' is not defined